In [89]:
import requests
import pandas as pd
from pprint import pprint
import time
import json
import csv
import math
import numpy

In [90]:
cdc500 = pd.read_csv("500_Cities__Local_Data_for_Better_Health__2018_release.csv", encoding="utf-8")
cdc5002=cdc500[(cdc500['GeographicLevel'] == 'City')&(cdc500['DataValueTypeID'] == 'AgeAdjPrv')&(cdc500['Measure'] == "Visits to doctor for routine checkup within the past Year among adults aged >=18 Years")]
cdc5002['PerYrlyDoc']=cdc5002['Data_Value']
cdcdoc = cdc5002[['StateAbbr','CityName','PerYrlyDoc']]


cdcdoc.head(5)


C:\Users\silva\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,StateAbbr,CityName,PerYrlyDoc
860,AL,Birmingham,76.9
3099,AL,Hoover,72.7
4246,AL,Huntsville,69.1
6191,AL,Mobile,74.1
8285,AL,Montgomery,76.7


In [91]:
#summarize by month by city, then average# of international arrivals by month by city

fldf= pd.read_csv("US International Flight Arrival 2018.csv", encoding="utf-8",usecols=['PASSENGERS','DEST','DEST_CITY_NAME','YEAR','MONTH'])
fldf[['City','State']] = fldf['DEST_CITY_NAME'].str.split(', ',expand=True)
fl= fldf.groupby(['City','State'])[["PASSENGERS"]].sum()
fl

,,PASSENGERS
City,State,
Abilene,TX,6
Aguadilla,PR,905
Akron,OH,21
Alamogordo,NM,0
Albany,NY,342
Albuquerque,NM,672
Alexandria,LA,2714
Allentown/Bethlehem/Easton,PA,7
Altoona,PA,1


In [92]:
cdcp=pd.read_csv("NationalOutbreakPersonToPerson.csv", encoding="utf-8")
cdcp=cdcp[(cdcp['Etiology Status'] == 'Confirmed')&(cdcp['Year'] == 2017)]
cdcpr= cdcp.groupby(['State'])[["Illnesses"]].sum()

cdcpr


,Illnesses
State,
Alabama,688
Alaska,126
Arizona,239
Colorado,54
Connecticut,86
Florida,32
Georgia,25
Hawaii,48
Illinois,3191


In [93]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
states=pd.DataFrame.from_dict(us_state_abbrev,orient='index', columns=['abrev'])
states.reset_index(level=0, inplace=True)
states=states.rename(columns={states.columns[0]:'name'})
states.head()

,name,abrev
0,Mississippi,MS
1,Oklahoma,OK
2,Delaware,DE
3,Minnesota,MN
4,Illinois,IL


In [94]:
sub1 = pd.merge(cdcpr, states,  how='left', left_on=['State'], right_on = ['name'])
sub1

,Illnesses,name,abrev
0,688,Alabama,AL
1,126,Alaska,AK
2,239,Arizona,AZ
3,54,Colorado,CO
4,86,Connecticut,CT
5,32,Florida,FL
6,25,Georgia,GA
7,48,Hawaii,HI
8,3191,Illinois,IL
9,1247,Indiana,IN


In [95]:
sub2 = pd.merge(cdcdoc, sub1,  how='left', left_on=['StateAbbr'], right_on = ['abrev'])
sub2

,StateAbbr,CityName,PerYrlyDoc,Illnesses,name,abrev
0,AL,Birmingham,76.9,688.0,Alabama,AL
1,AL,Hoover,72.7,688.0,Alabama,AL
2,AL,Huntsville,69.1,688.0,Alabama,AL
3,AL,Mobile,74.1,688.0,Alabama,AL
4,AL,Montgomery,76.7,688.0,Alabama,AL
5,AL,Tuscaloosa,74.7,688.0,Alabama,AL
6,AK,Anchorage,58.8,126.0,Alaska,AK
7,AZ,Avondale,66.2,239.0,Arizona,AZ
8,AZ,Chandler,66.9,239.0,Arizona,AZ
9,AZ,Gilbert,66.9,239.0,Arizona,AZ


In [96]:
sub3 = pd.merge(sub2, fl,  how='left', left_on=['StateAbbr','CityName'], right_on = ['State','City'])
sub3



,StateAbbr,CityName,PerYrlyDoc,Illnesses,name,abrev,PASSENGERS
0,AL,Birmingham,76.9,688.0,Alabama,AL,971.0
1,AL,Hoover,72.7,688.0,Alabama,AL,NaN
2,AL,Huntsville,69.1,688.0,Alabama,AL,19.0
3,AL,Mobile,74.1,688.0,Alabama,AL,NaN
4,AL,Montgomery,76.7,688.0,Alabama,AL,NaN
5,AL,Tuscaloosa,74.7,688.0,Alabama,AL,0.0
6,AK,Anchorage,58.8,126.0,Alaska,AK,35869.0
7,AZ,Avondale,66.2,239.0,Arizona,AZ,NaN
8,AZ,Chandler,66.9,239.0,Arizona,AZ,NaN
9,AZ,Gilbert,66.9,239.0,Arizona,AZ,NaN


In [98]:
sub3.sort_values(by='PASSENGERS', ascending=False)

,StateAbbr,CityName,PerYrlyDoc,Illnesses,name,abrev,PASSENGERS
342,NY,New York,74.2,132.0,New York,NY,17765774.0
74,CA,Los Angeles,66.2,NaN,NaN,NaN,12563882.0
186,FL,Miami,74.7,32.0,Florida,FL,10510175.0
222,IL,Chicago,70.9,3191.0,Illinois,IL,7206592.0
112,CA,San Francisco,65.0,NaN,NaN,NaN,6975787.0
329,NJ,Newark,80.2,NaN,NaN,NaN,6811083.0
204,GA,Atlanta,76.6,25.0,Georgia,GA,6139421.0
429,TX,Houston,69.3,354.0,Texas,TX,5721170.0
177,FL,Fort Lauderdale,76.1,32.0,Florida,FL,4258984.0
168,DC,Washington,80.0,NaN,NaN,NaN,4118370.0
